#### Assi-7. Write a program to implement CNN using image dataset.

In [2]:
# prompt: extract faces.zip

import zipfile
import urllib.request

# Extract the contents of the zip file
with zipfile.ZipFile("Datasets\\faces.zip", 'r') as zip_ref:
    zip_ref.extractall("faces")

In [1]:
%pip install tensorflow

  Using cached tensorflow-2.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.28.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached grpcio-1.68.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metada

In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pandas as pd
from sklearn.model_selection import train_test_split
import os


# Load the CSV file
df = pd.read_csv('faces/train.csv')

# Define the directory containing the images
image_dir = 'faces/Train/'

# Split the data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Define ImageDataGenerator for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=image_dir,
    x_col='ID',
    y_col='Class',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=image_dir,
    x_col='ID',
    y_col='Class',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 3286 validated image filenames belonging to 3 classes.
Found 841 validated image filenames belonging to 3 classes.


/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 4676 invalid image filename(s) in x_col="ID". These filename(s) will be ignored.
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1150 invalid image filename(s) in x_col="ID". These filename(s) will be ignored.
  warnings.warn(


In [25]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'), 
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

# Use a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [19]:
import pandas as pd
import os
import random

# Read the CSV file
df = pd.read_csv('faces/train.csv')

# Get list of all image files
image_files = df['ID'].tolist()

# Randomly select half of the files to delete
files_to_delete = random.sample(image_files, len(image_files)//2)

# Delete selected files
for file in files_to_delete:
    file_path = os.path.join('faces/Train', file)
    if os.path.exists(file_path):
        os.remove(file_path)

# Update the CSV by removing entries of deleted files
df_updated = df[~df['ID'].isin(files_to_delete)]

# Save the updated CSV
df_updated.to_csv('faces/train.csv', index=False)

In [26]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=20
)

Epoch 1/20
  3/102 ━━━━━━━━━━━━━━━━━━━━ 2:29 2s/step - accuracy: 0.4861 - loss: 1.0783

KeyboardInterrupt: 

In [24]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 131ms/step - accuracy: 0.6661 - loss: 0.7647
Validation Loss: 0.7745416760444641
Validation Accuracy: 0.6777645945549011


In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale the image
    return img_array

# Function to make predictions
def predict_image(img_path, model):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class

# Example usage
img_path = 'image.png'
predicted_class = predict_image(img_path, model)

# Map the predicted class index to the class label
class_labels = {0: 'YOUNG', 1: 'MIDDLE', 2: 'OLD'}
predicted_label = class_labels[predicted_class[0]]

print(f'Predicted class: {predicted_label}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Predicted class: YOUNG
